In [123]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import sklearn as skl
import tensorflow as tf


In [124]:
df = pd.read_csv("Data/carsAvgPrice.csv")
df.head()

,Unnamed: 0,Brand,Model,Year,Status,Mileage,Age,AveragePrice
0,0,Acura,ILX,2013,Used,86557.0,11,13900.0
1,1,Acura,ILX,2014,Used,148266.0,10,10996.0
2,2,Acura,ILX,2015,Used,77223.0,9,13650.0
3,3,Acura,ILX,2016,Used,56546.0,8,16900.0
4,4,Acura,ILX,2016,Used,72301.0,8,16999.0


In [125]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Brand,Model,Year,Status,Mileage,Age,AveragePrice
0,Acura,ILX,2013,Used,86557.0,11,13900.0
1,Acura,ILX,2014,Used,148266.0,10,10996.0
2,Acura,ILX,2015,Used,77223.0,9,13650.0
3,Acura,ILX,2016,Used,56546.0,8,16900.0
4,Acura,ILX,2016,Used,72301.0,8,16999.0


In [126]:
brandCount = df["Brand"].value_counts()
brandCount[brandCount<1000]

Brand
Subaru                 982
Volvo                  916
Tesla                  881
Lincoln                867
Infiniti               817
RAM                    775
Genesis                516
Jaguar                 331
Buick                  324
Chrysler               307
Mitsubishi             279
Maserati               191
MINI                   181
Alfa Romeo             149
Bentley                119
Pontiac                105
Ferrari                 93
Lamborghini             85
FIAT                    77
Rolls-Royce             64
Aston Martin            61
Scion                   52
Hummer                  47
Polestar                45
McLaren                 40
Rivian                  32
Saturn                  30
Mercury                 18
Lucid                   13
Saab                    13
Lotus                   11
Smart                    8
Suzuki                   6
Karma                    6
Plymouth                 4
Oldsmobile               3
International Scout   

In [127]:
brands_to_replace = list(brandCount[brandCount<1200].index)

for brand in brands_to_replace:
    df['Brand'] = df['Brand'].replace(brand,"Other")

df.drop(df.loc[df['Brand']=='Other'].index,inplace=True)
#df.drop(df.loc[df['Brand']=='Porsche'].index,inplace=True)
df["Brand"].value_counts()

Brand
Toyota        6461
Ford          6124
Chevrolet     4546
BMW           3506
Lexus         3061
Honda         2665
Mercedes      2606
Audi          2385
Jeep          2054
GMC           1949
Porsche       1691
Nissan        1578
Cadillac      1342
Kia           1273
Volkswagen    1212
Mazda         1208
Name: count, dtype: int64

In [128]:
modelCount = df["Model"].value_counts()
modelCount[modelCount<500]

Model
Camaro      485
ES          436
Bronco      434
CR-V        432
Explorer    420
           ... 
550i          1
G20           1
LTD           1
Prelude       1
Rondo         1
Name: count, Length: 365, dtype: int64

In [129]:
models_to_replace = list(modelCount[modelCount<500].index)

for model in models_to_replace:
    df['Model'] = df['Model'].replace(model,"Other")

df.drop(df.loc[df['Model']=='Other'].index,inplace=True)
df["Model"].value_counts()

Model
F-150             1796
Corvette           954
4Runner            940
Tundra             932
Mustang            871
Wrangler           807
Yukon              801
Silverado          798
RX                 774
Tacoma             773
Sierra             760
Grand Cherokee     744
Tahoe              720
Highlander         694
F-250              653
RAV4               650
Expedition         582
Camry              557
X5                 555
911                555
Accord             554
Escalade           533
Civic              512
GX                 500
Name: count, dtype: int64

In [130]:
df_dummy = pd.get_dummies(df,dtype=int)
df_dummy.head()

,Year,Mileage,Age,AveragePrice,Brand_BMW,Brand_Cadillac,Brand_Chevrolet,Brand_Ford,Brand_GMC,Brand_Honda,...,Model_Silverado,Model_Tacoma,Model_Tahoe,Model_Tundra,Model_Wrangler,Model_X5,Model_Yukon,Status_Certified,Status_New,Status_Used
6062,2004,44671.0,20,12500.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6063,2009,108871.0,15,9495.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6064,2010,63146.0,14,8900.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6065,2011,39873.0,13,16850.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6066,2011,46114.0,13,27588.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [131]:
y = df_dummy['AveragePrice']
X = df_dummy.drop(columns=['AveragePrice','Year'])

In [132]:
y

6062     12500.000000
6063      9495.000000
6064      8900.000000
6065     16850.000000
6066     27588.000000
             ...     
54248    48490.000000
54249    50849.000000
54250    58492.000000
54251    64771.000000
54252    64178.888889
Name: AveragePrice, Length: 18015, dtype: float64

In [133]:
X

,Mileage,Age,Brand_BMW,Brand_Cadillac,Brand_Chevrolet,Brand_Ford,Brand_GMC,Brand_Honda,Brand_Jeep,Brand_Lexus,...,Model_Silverado,Model_Tacoma,Model_Tahoe,Model_Tundra,Model_Wrangler,Model_X5,Model_Yukon,Status_Certified,Status_New,Status_Used
6062,44671.0,20,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6063,108871.0,15,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6064,63146.0,14,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6065,39873.0,13,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6066,46114.0,13,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54248,33696.0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
54249,105.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
54250,186.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
54251,1455.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [134]:
#Train Test split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [135]:
scaler = StandardScaler()

In [136]:
#scale data
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [137]:
#model Random Forest  
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=50, random_state=13)
rf_model = rf_model.fit(X_train_scaled,y_train)
predictions = rf_model.predict(X_test_scaled)

In [138]:
predictions_df = X_test.copy()
predictions_df["Actual_price"] = y_test
predictions_df["Predict_price"] = predictions
predictions_df["Error %"] = (abs(predictions_df["Predict_price"]-predictions_df["Actual_price"])/predictions_df["Actual_price"])*100
predictions_df.head()

,Mileage,Age,Brand_BMW,Brand_Cadillac,Brand_Chevrolet,Brand_Ford,Brand_GMC,Brand_Honda,Brand_Jeep,Brand_Lexus,...,Model_Tundra,Model_Wrangler,Model_X5,Model_Yukon,Status_Certified,Status_New,Status_Used,Actual_price,Predict_price,Error %
12980,15938.0,3,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,65499.0,58676.82,10.415701
25228,8577.0,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,29522.0,30322.74,2.712350
17086,111628.0,11,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,18999.0,14198.60,25.266593
22083,42025.0,3,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,38535.0,47471.74,23.191229
50267,158822.0,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,17990.0,20668.40,14.888271


In [139]:
predictions_df.loc[:,'Error %'].mean()

15.372517905196144

In [140]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
r2 = r2_score(predictions_df["Actual_price"], predictions_df["Predict_price"])
mse = mean_squared_error(predictions_df["Actual_price"], predictions_df["Predict_price"])
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The r2 is 0.7990187551454767.
The mean squared error is 162145531.86677262.
The root mean squared error is 12733.637809627406.
The standard deviation is 30696.854360265348.


In [141]:
result_df = pd.merge(predictions_df,df,how='left',left_index=True,right_index=True)
result_df = result_df[["Brand","Model","Year","Status","Mileage_y","AveragePrice","Actual_price","Predict_price","Error %"]]
result_df

,Brand,Model,Year,Status,Mileage_y,AveragePrice,Actual_price,Predict_price,Error %
12980,Chevrolet,Tahoe,2021,Certified,15938.0,65499.0,65499.0,58676.82,10.415701
25228,Honda,Civic,2022,Used,8577.0,29522.0,29522.0,30322.74,2.712350
17086,Ford,F-150,2013,Used,111628.0,18999.0,18999.0,14198.60,25.266593
22083,GMC,Sierra,2021,Used,42025.0,38535.0,38535.0,47471.74,23.191229
50267,Toyota,Highlander,2016,Used,158822.0,17990.0,17990.0,20668.40,14.888271
...,...,...,...,...,...,...,...,...,...
22215,GMC,Sierra,2022,Used,11533.0,61800.0,61800.0,61012.56,1.274175
16981,Ford,F-150,2011,Used,115349.0,16488.0,16488.0,19708.66,19.533358
49507,Toyota,Camry,2021,Certified,43079.0,27786.0,27786.0,27575.38,0.758008
51318,Toyota,RAV4,2017,Used,86905.0,17999.0,17999.0,20613.74,14.527140


In [142]:
import hvplot.pandas
result_df.hvplot.scatter(
    x='Actual_price',
    y='Predict_price',
    color='Brand'
)

:Scatter   [Actual_price]   (Predict_price,Brand)